In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
import time

In [0]:
spark = SparkSession.builder \
    .appName("ETL Analysis") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.4.0") \
    .getOrCreate()

In [0]:
def extract_spark(file_path):
    start_time = time.time()
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    duration = time.time() - start_time
    return df, duration

In [0]:
def transform_spark_rename_columns(df):
    start_time = time.time()
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, f"{col_name}_new")
    duration = time.time() - start_time
    return duration
    
def transform_spark_drop_na(df):
    start_time = time.time()
    df = df.dropna()
    duration = time.time() - start_time
    return duration

In [0]:
def get_file_size(path):
    file_info = dbutils.fs.ls(path)
    total_size = sum(f.size for f in file_info if f.isFile())
    return total_size

In [0]:
def load_spark_csv(df, output_path):
    start_time = time.time()
    df.write.csv(output_path, header=True, mode='overwrite')
    duration = time.time() - start_time

    file_size = get_file_size(output_path)

    return duration, file_size


def load_spark_delta(df, output_path):
    start_time = time.time()
    df.write.format("delta").mode("overwrite").save(output_path)
    duration = time.time() - start_time

    file_size = get_file_size(output_path)

    return duration, file_size


def load_spark_parquet(df, output_path):
    start_time = time.time()
    df.write.parquet(output_path, mode='overwrite')
    duration = time.time() - start_time

    file_size = get_file_size(output_path)

    return duration, file_size


def load_spark_avro(df, output_path):
    start_time = time.time()
    df.write.format("avro").mode("overwrite").save(output_path)
    duration = time.time() - start_time

    file_size = get_file_size(output_path)

    return duration, file_size


def load_spark_orc(df, output_path):

    start_time = time.time()
    df.write.format("orc").mode("overwrite").option("mergeSchema", "true").save(output_path)
    duration = time.time() - start_time

    file_size = get_file_size(output_path)

    return duration, file_size


In [0]:
datasets = ['transactions_data.csv', 'titanic.csv', 'reviews.csv', 'locations.csv']

dim_datasets = []
fact_metrics = []

for index, dataset in enumerate(datasets):
    primary_key = index + 1
    path = dataset.split('.')[0]
    
    df_raw, extract_time = extract_spark(f'dbfs:/FileStore/bigdata/{dataset}')
    
    transform_rename_columns = transform_spark_rename_columns(df_raw)
    transform_dropna = transform_spark_drop_na(df_raw)
    
    load_time_csv, file_size_csv = load_spark_csv(df_raw, f"/dbfs/FileStore/bigdata/output_spark/{path}/csv")
    load_time_delta, file_size_delta = load_spark_delta(df_raw, f"/dbfs/FileStore/bigdata/output_spark/{path}/delta")
    load_time_parquet, file_size_parquet = load_spark_parquet(df_raw, f"/dbfs/FileStore/bigdata/output_spark/{path}/parquet")
    load_time_avro, file_size_avro = load_spark_avro(df_raw, f"/dbfs/FileStore/bigdata/output_spark/{path}/avro")
    load_time_orc, file_size_orc = load_spark_orc(df_raw, f"/dbfs/FileStore/bigdata/output_orc/{path}/orc")
    
    dim_datasets.append({
        "id": primary_key,
        "dataset_name": path,
        "number_of_rows": df_raw.count()
    })
    
    fact_metrics.append({
        "dataset_id": primary_key,
        "extract_time": round(extract_time,2),
        "transform_rename_columns_time": round(transform_rename_columns,2),
        "transform_dropna_time": round(transform_dropna,2),
        "load_time_csv": round(load_time_csv,2),
        "file_size_csv_mb": round(file_size_csv / (1024 * 1024),2),
        "load_time_delta": round(load_time_delta,2),
        "file_size_delta_mb": round(file_size_delta / (1024 * 1024),2),
        "load_time_parquet": round(load_time_parquet,2),
        "file_size_parquet_mb": round(file_size_parquet / (1024 * 1024),2),
        "load_time_avro": round(load_time_avro,2),
        "file_size_avro_mb": round(file_size_avro / (1024 * 1024),2),
        "load_time_orc": round(load_time_orc,2),
        "file_size_orc_mb": round(file_size_orc / (1024 * 1024),2)
    })
    


dim_datasets_df = spark.createDataFrame(dim_datasets)
fact_metrics_df = spark.createDataFrame(fact_metrics)

dim_datasets_df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/bigdata/output_spark/dim_datasets")
fact_metrics_df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/bigdata/output_spark/fact_metrics_pyspark")

print("Process finished successfully!")

Process finished successfully!


In [0]:
dim_datasets_df = spark.read.format("delta").load("/dbfs/FileStore/bigdata/output_spark/dim_datasets")
dim_datasets_df.display()

dataset_name,id,number_of_rows
transactions_data,1,13305915
locations,4,845
titanic,2,891
reviews,3,703796


In [0]:
fact_metrics_df = spark.read.format("delta").load("/dbfs/FileStore/bigdata/output_spark/fact_metrics_pyspark")
fact_metrics_df.display()

dataset_id,extract_time,file_size_avro_mb,file_size_csv_mb,file_size_delta_mb,file_size_orc_mb,file_size_parquet_mb,load_time_avro,load_time_csv,load_time_delta,load_time_orc,load_time_parquet,transform_dropna_time,transform_rename_columns_time
1,87.13,456.16,1263.69,4164.48,267.84,244.98,127.41,181.14,206.78,131.53,129.65,0.05,0.36
4,1.15,0.18,0.35,2.36,0.17,0.18,1.69,1.96,10.11,2.16,2.09,0.02,0.17
2,2.38,0.04,0.06,0.45,0.03,0.04,2.07,2.52,11.58,2.26,2.39,0.02,0.15
3,5.99,3.32,14.56,19.67,1.69,1.52,4.75,6.28,13.65,4.75,4.82,0.01,0.02


In [0]:
df_final = fact_metrics_df \
    .join(
        dim_datasets_df.select("id", "dataset_name", "number_of_rows"), 
        fact_metrics_df["dataset_id"] == dim_datasets_df["id"],
        how="left" 
    )

df_final.display()

dataset_id,extract_time,file_size_avro_mb,file_size_csv_mb,file_size_delta_mb,file_size_orc_mb,file_size_parquet_mb,load_time_avro,load_time_csv,load_time_delta,load_time_orc,load_time_parquet,transform_dropna_time,transform_rename_columns_time,id,dataset_name,number_of_rows
1,87.13,456.16,1263.69,4164.48,267.84,244.98,127.41,181.14,206.78,131.53,129.65,0.05,0.36,1,transactions_data,13305915
4,1.15,0.18,0.35,2.36,0.17,0.18,1.69,1.96,10.11,2.16,2.09,0.02,0.17,4,locations,845
2,2.38,0.04,0.06,0.45,0.03,0.04,2.07,2.52,11.58,2.26,2.39,0.02,0.15,2,titanic,891
3,5.99,3.32,14.56,19.67,1.69,1.52,4.75,6.28,13.65,4.75,4.82,0.01,0.02,3,reviews,703796


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.